# imports

In [5]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import *

from Model import HourglassNet
from Data_utils import DataGenerator

## model

In [6]:
model = HourglassNet(num_classes=7, 
                     num_stacks=1, 
                     num_channels=128, 
                     inres=(128,128), 
                     outres=(32,32)).get_model()

In [7]:
def mse_4x_penalty(y_true, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)*4
    a = K.less(K.max(K.argmax(y_true[:,:,:,2], axis=2)), K.max(K.argmax(y_pred[:,:,:,0], axis=2)))
    b = K.less(K.max(K.argmax(y_pred[:,:,:,1], axis=2)), K.max(K.argmax(y_true[:,:,:,2], axis=2)))
    cond = tf.cond(
        K.any(K.stack([a, b], axis=0), axis=0),
        lambda: mse*10,
        lambda: mse
    )
    return cond

def mae_4x(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_error(y_true, y_pred)*4

In [8]:
optimizer = tf.keras.optimizers.Adam(lr=5e-4)
model.compile(optimizer=optimizer, loss=mse_4x_penalty, metrics=[mae_4x])

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
front_conv_1x1_x1 (Conv2D)      (None, 64, 64, 64)   3200        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 64, 64, 64)   256         front_conv_1x1_x1[0][0]          
__________________________________________________________________________________________________
front_residual_x1_conv_1x1_x1 ( (None, 64, 64, 32)   2080        batch_normalization_53[0][0]     
__________________________________________________________________________________________________
batch_norm

## datasets

In [10]:
train_gen = DataGenerator('dataset/', 'train_landmarks.csv', 
                          inres=(128,128), outres=(32,32), nparts=7, is_train=True)
valid_gen = DataGenerator('dataset/', 'valid_landmarks.csv', 
                          inres=(128,128), outres=(32,32), nparts=7)
test_gen = DataGenerator('dataset/', 'test_landmarks.csv', 
                         inres=(128,128), outres=(32,32), nparts=7)

## callbacks

In [11]:
my_call = [
    ModelCheckpoint('train/model.h5', save_best_only=True),
    EarlyStopping(patience=40),
    TensorBoard(log_dir='train/'),
    ReduceLROnPlateau(patience=20),
    CSVLogger('train/history.csv')
]

## training

In [12]:
batch_size = 32
n_stack = 1
epochs = 1000
sigma = 1.0

model.fit_generator(train_gen.generator(batch_size, n_stack, sigma=sigma), len(train_gen)//batch_size,
                    validation_data=valid_gen.generator(batch_size, n_stack, sigma=sigma), 
                    validation_steps=len(valid_gen)//batch_size,
                    epochs=epochs, 
                    callbacks=my_call)

Epoch 1/1000
83/83 [==============================] - 28s 338ms/step - loss: 361.0574 - mae_4x: 1.3191 - val_loss: 1449.3931 - val_mae_4x: 3.2746
Epoch 2/1000
83/83 [==============================] - 9s 108ms/step - loss: 28.6250 - mae_4x: 0.4375 - val_loss: 3326.4771 - val_mae_4x: 4.8439
Epoch 3/1000
83/83 [==============================] - 9s 114ms/step - loss: 16.6540 - mae_4x: 0.3244 - val_loss: 1020.3224 - val_mae_4x: 2.0693
Epoch 4/1000
83/83 [==============================] - 9s 113ms/step - loss: 12.1424 - mae_4x: 0.2706 - val_loss: 172.8026 - val_mae_4x: 0.8207
Epoch 5/1000
83/83 [==============================] - 9s 114ms/step - loss: 9.2370 - mae_4x: 0.2272 - val_loss: 28.4455 - val_mae_4x: 0.4315
Epoch 6/1000
83/83 [==============================] - 9s 108ms/step - loss: 7.7203 - mae_4x: 0.2012 - val_loss: 84.4099 - val_mae_4x: 0.5632
Epoch 7/1000
83/83 [==============================] - 9s 108ms/step - loss: 6.7589 - mae_4x: 0.1828 - val_loss: 94.4071 - val_mae_4x: 0.6901


Epoch 59/1000
83/83 [==============================] - 9s 108ms/step - loss: 2.7542 - mae_4x: 0.0755 - val_loss: 7.5047 - val_mae_4x: 0.1764
Epoch 60/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.6139 - mae_4x: 0.0695 - val_loss: 3.1010 - val_mae_4x: 0.0922
Epoch 61/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.4660 - mae_4x: 0.0645 - val_loss: 2.7464 - val_mae_4x: 0.0725
Epoch 62/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.3773 - mae_4x: 0.0618 - val_loss: 2.6498 - val_mae_4x: 0.0672
Epoch 63/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.3046 - mae_4x: 0.0600 - val_loss: 2.5866 - val_mae_4x: 0.0654
Epoch 64/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.2430 - mae_4x: 0.0585 - val_loss: 2.5344 - val_mae_4x: 0.0643
Epoch 65/1000
83/83 [==============================] - 9s 109ms/step - loss: 2.1907 - mae_4x: 0.0571 - val_loss: 2.5187 - val_mae_4x: 0.0632
Epoch 66/1000

Epoch 117/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.1060 - mae_4x: 0.0338 - val_loss: 2.7800 - val_mae_4x: 0.0444
Epoch 118/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.0980 - mae_4x: 0.0338 - val_loss: 2.8466 - val_mae_4x: 0.0446
Epoch 119/1000
83/83 [==============================] - 9s 109ms/step - loss: 1.0894 - mae_4x: 0.0337 - val_loss: 2.8180 - val_mae_4x: 0.0447
Epoch 120/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.0852 - mae_4x: 0.0337 - val_loss: 2.8248 - val_mae_4x: 0.0451
Epoch 121/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.0786 - mae_4x: 0.0337 - val_loss: 2.8044 - val_mae_4x: 0.0446
Epoch 122/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.0770 - mae_4x: 0.0337 - val_loss: 2.8815 - val_mae_4x: 0.0449
Epoch 123/1000
83/83 [==============================] - 9s 108ms/step - loss: 1.0701 - mae_4x: 0.0336 - val_loss: 2.8231 - val_mae_4x: 0.0457


## evaluation

In [14]:
model.evaluate_generator(test_gen.generator(1,n_stack,sigma=sigma), steps=len(test_gen))

[0.0859802779959864, 0.04566506155242678]